In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95!important; }</style>"))

In [3]:
# Instalando bibliotecas
!pip install scrapy

You are using pip version 18.1, however version 20.0.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [4]:
import requests
from scrapy.http import TextResponse

In [32]:
def send_request_to_url(site_url):
    # py_res --> Response nativa do Python
    py_res = requests.get(site_url)
    #print(py_res)

    # scrapy_res --> Response própria para ser utilizada no scrapy
    scrapy_res = TextResponse(py_res.url, body=py_res.text, encoding='utf-8')
    #print(scrapy_res)
    
    return scrapy_res

In [33]:
url = 'https://rebep.org.br/revista/issue/view/'

In [34]:
import time
# Navega pelo website coletando os trabalhos da RBEP
# texts é uma lista que irá salvar dicionários
# cada dicionário é uma revista da RBEP
texts = []

# iteramos sobre as paginas - verificação manual do número de páginas
for i in range(1,73):
    # observacao: estamos fazendo requests em um loop. Adicionaremos um delay
    scrapy_res = send_request_to_url(url + str(i))
    
    # Delay de 1s
    time.sleep(1)
    
    # exploramos a páginas web a partir das "ferramentas do desenvolvedor"
    # abrimos a páginas web que queremos explorar e 
    # fazemos Ctrl + Shift + I
    for sel in scrapy_res.xpath('//div[@class="section"]'):
        # Dentro do loop, fazemos o parsing das tags filhas do div corrente
        text_dict = {}
        
        text_dict['link'] = url + str(i)
        
        text_dict['edicao'] = scrapy_res.xpath('//div[@class="page page_issue"]//nav[@class="cmp_breadcrumbs"]//ol//li[@class="current"]//h1/text()').extract()
        
        # Le o tipo de trabalho (Tag h2, classe "text")
        text_dict['tipo_trabalho'] = (sel
                                      .xpath('.//h2/text()')
                                      .extract_first())
        
        # Le o título do trabalho
        text_dict['titulo'] = (sel
                               .xpath('.//div[@class="title"]/a/text()')
                               .extract())
        # Le os autores do trabalho
        text_dict['autor'] = (sel
                              .xpath('.//div[@class="authors"]/text()')
                              .extract())
        # Le o número de páginas do trabalho
        text_dict['paginas'] = (sel
                                .xpath('.//div[@class="pages"]/text()')
                                .extract())
        
        texts.append(text_dict)

In [35]:
texts

[{'autor': ['\n\t\t\tNeide Lopes Patarra\n\t\t'],
  'edicao': ['\n\t\t\t\t\t\t\t\t\tv. 1 n. 1/2 (1984)\n\t\t\t\t\t\t\t'],
  'link': 'https://rebep.org.br/revista/issue/view/1',
  'paginas': [],
  'tipo_trabalho': '\n\t\t\t\t\tEditorial\n\t\t\t\t',
  'titulo': ['\n\t\t\tApresentação\n\t\t\t\t\t']},
 {'autor': ['\n\t\t\tDonald R. Saywer\n\t\t',
   '\n\t\t\tElza Berquó, Maria Andréa Loyola\n\t\t',
   '\n\t\t\tGeorge Martine, Liscio Camargo\n\t\t',
   '\n\t\t\tLuis Patricio Ortiz, Lucia Mayumi Yazaki\n\t\t',
   '\n\t\t\tRalph Hakkert\n\t\t'],
  'edicao': ['\n\t\t\t\t\t\t\t\t\tv. 1 n. 1/2 (1984)\n\t\t\t\t\t\t\t'],
  'link': 'https://rebep.org.br/revista/issue/view/1',
  'paginas': ['\n\t\t\t\t3-34\n\t\t\t',
   '\n\t\t\t\t35-98\n\t\t\t',
   '\n\t\t\t\t99-144\n\t\t\t',
   '\n\t\t\t\t145-170\n\t\t\t',
   '\n\t\t\t\t171-192\n\t\t\t'],
  'tipo_trabalho': '\n\t\t\t\t\tArtigos originais\n\t\t\t\t',
  'titulo': ['\n\t\t\tFluxo e Refluxo da Fronteira agrícola no Brasil: ensaio de interpretação estru

In [36]:
import pandas as pd

In [37]:
# problema: transformar um dicionario com itens de tamanhos diferentes em um data frame
# solução
# https://stackoverflow.com/questions/19736080/creating-dataframe-from-a-dictionary-where-entries-have-different-lengths
# "problema", que é resolvível, como os dicionários tem tamanhos diferentes,
# ao criar o data frame apenas a primeira linha tem conteúdo, as demais ficam com NaN

texts_df = []
for i in range(0,len(texts)):
    df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in texts[i].items() ]))
    texts_df.append(df)

In [39]:
pd.concat(texts_df,ignore_index=True).to_csv('trabalhos_rbep.csv')